In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai import * 
from fastai.text import *
from pathlib import Path 
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [3]:
pip install torchtext

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils


In [5]:
path = Config.data_path()
path.mkdir(parents=True, exist_ok=True)

path.ls()

In [6]:
df_train = pd.read_csv(path/'train.csv')
df_test = pd.read_csv(path/'test.csv')
# df_train
print(df_train.shape)
print(len(df_train))

(1306122, 3)
1306122


In [7]:
import string
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]



mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def mispelled_words(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = mispelled_words(mispell_dict)

def replace_typical_mispell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace,text)

def clean_text(x):

    x = str(x)
    for punct in puncts:
        x.translate(str.maketrans('', '', string.punctuation))
    return x



In [8]:
len(df_train)


1306122

In [9]:
df_train["question_text"] = df_train["question_text"].str.lower()
df_test["question_text"] = df_test["question_text"].str.lower()

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: clean_text(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x: clean_text(x))

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: clean_numbers(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x:clean_numbers(x))

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: replace_typical_mispell(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x: replace_typical_mispell(x))

train_x = df_train['question_text'].fillna('_##_').values
test_x = df_test['question_text'].fillna('_##_').values

my bar!: 100%|██████████| 375806/375806 [00:05<00:00, 73500.20it/s]


import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [10]:
train_length = len(df_train)
print(train_length)
train_length*0.80 

1306122


1044897.6000000001

In [13]:
# train_y = df_train['target'].values

In [14]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = "/Users/caitlinaspinall/.fastai/data/train.csv",
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT), ("target", LABEL)] 
    test = torchtext.data.TabularDataset( 
                path="/Users/caitlinaspinall/.fastai/data/test.csv",
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [20]:
TEXT,LABEL,train,test = prepare_sequences()


In [21]:
TEXT.build_vocab(train, test, vectors = "glove.6B.100d")

In [22]:

from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size=64, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)
test_iter = Iterator(test, batch_size = 64, device = None, sort = False,  sort_within_batch = False, repeat = False) #Changed device from -1 to None
# val_iter = Iterator(val, batch_size = 64, device = None, sort = False,  sort_within_batch = False, repeat = False)

In [60]:
iter(train_iter).__next__().question_text
iter(train_iter).__next__().question_text.shape

val,_ = train.split(0.2)


In [65]:
print(len(val))
len(train)


261224


1306122

In [24]:
TEXT.vocab.vectors.shape

torch.Size([527626, 100])

In [37]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        hidden_size = 64
        self.embedding = nn.Embedding(len(TEXT.vocab),100)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        self.lstm = nn.LSTM(100,hidden_size, bidirectional = True, batch_first = True)
        
        self.linear = nn.Linear(hidden_size * 4, 2)
        self.relu = nn.ReLU()
        self.out = nn.Linear(16,1)
        
    def forward(self,train):
        h_embedding = self.embedding(train)
#         h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding,0)))
            
        print(h_embedding.shape)  
        h_lstm, _ = self.lstm(h_embedding)
            
        avg_pool = torch.mean(h_lstm,1)
        print(avg_pool.shape)
        max_pool,_ = torch.max(h_lstm,1)
        
        conc = torch.cat((avg_pool, max_pool),1)
        print(conc.shape)
        conc = self.relu(self.linear(conc))
        print(conc.shape)
        
#         out = self.out(conc)
        out = nn.LogSoftmax(dim=1)(conc)
        return out
            
            
batch = next(iter(train_iter)).question_text 
net = NeuralNet()  
net(batch)

torch.Size([52, 64, 100])
torch.Size([52, 128])
torch.Size([52, 256])
torch.Size([52, 2])


tensor([[-0.6656, -0.7214],
        [-0.6569, -0.7307],
        [-0.6348, -0.7551],
        [-0.6598, -0.7277],
        [-0.6434, -0.7455],
        [-0.6311, -0.7593],
        [-0.6320, -0.7583],
        [-0.6525, -0.7355],
        [-0.6602, -0.7273],
        [-0.6580, -0.7296],
        [-0.6606, -0.7268],
        [-0.6593, -0.7282],
        [-0.6917, -0.6946],
        [-0.6638, -0.7234],
        [-0.6717, -0.7150],
        [-0.6581, -0.7294],
        [-0.6806, -0.7058],
        [-0.6803, -0.7062],
        [-0.6896, -0.6967],
        [-0.6844, -0.7020],
        [-0.6931, -0.6931],
        [-0.6838, -0.7026],
        [-0.6931, -0.6931],
        [-0.6703, -0.7166],
        [-0.6931, -0.6931],
        [-0.6883, -0.6981],
        [-0.6931, -0.6931],
        [-0.6931, -0.6931],
        [-0.6931, -0.6931],
        [-0.6931, -0.6931],
        [-0.6931, -0.6931],
        [-0.6928, -0.6935],
        [-0.6931, -0.6931],
        [-0.6886, -0.6977],
        [-0.6931, -0.6931],
        [-0.6931, -0

In [38]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [59]:
def train_model(train_data, model,num_epochs, show = False):
    errors = []
    eval_errors = []
    f1_train = []
    auc_eval = []
    f1_eval =[]
    auc_eval=[]
    num_training_batches = len(train_data)

    optimizer = optim.SGD(model.parameters(),lr= 0.01)    
    loss_function = nn.NLLLoss()

    for epoch in range(num_epochs):
        model.train()
        train_iter = iter(train_data)
        model.zero_grad()
        train_loss = 0
        all_preds = []
        all_actual = []
        
        for batch_num in range(num_training_batches):
            batch = next(train_iter)
            train_batch, class_vector = batch.question_text.transpose(0,1),batch.target.unsqueeze(1)
            
            optimizer.zero_grad()
            preds = model(train_batch)
            loss = loss_function(preds,class_vector.reshape(64,1).squeeze(1))            


            train_loss += loss.item()
            loss.backward()
            optimizer.step()
#             train_avg_loss = train_loss/(num_training_batchsize*batch_size)
#         train_confusion_matriz = confusion_matric(all_actaul, all_preds).ravel()
        
#         f1_train.append(f1_score(all_actual,all_preds))
#         errors.append(train_loss_avg)
#         model.eval()
#         num_val_batches = len(val)
#         val_iter = iter(val)
#         eval_loss =0
#         all_pred = []
#         all_actual = []
        
   

train_model(train_iter, net, 3, show = False )  

torch.Size([64, 30, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 48, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 35, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 34, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 39, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 41, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 43, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 37, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 28, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 39, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 42, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size

torch.Size([64, 47, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 45, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 42, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 39, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 26, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 44, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 37, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 33, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 45, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 38, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size

torch.Size([64, 43, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 46, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 46, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 33, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 30, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 46, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size

torch.Size([64, 41, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 44, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 46, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 41, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 31, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 41, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 41, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 36, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 43, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 30, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 33, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size

torch.Size([64, 27, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 33, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 36, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 36, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 33, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 42, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 43, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 37, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 48, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size

torch.Size([64, 44, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 36, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 45, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 28, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 35, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 30, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 37, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 30, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 33, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 35, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 29, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size

torch.Size([64, 30, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 44, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 39, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 37, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 40, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 26, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 43, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 23, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 41, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])
torch.Size([64, 43, 100])
torch.Size([64, 128])
torch.Size([64, 256])
torch.Size([64, 2])


KeyboardInterrupt: 

In [ ]:
df_train.shape
print()

In [ ]:
1306122 * 0.2

In [ ]:
1306122-261224